# Simplex
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Resumo" data-toc-modified-id="Resumo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Resumo</a></span></li><li><span><a href="#Implementação" data-toc-modified-id="Implementação-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementação</a></span></li><li><span><a href="#Exemplos" data-toc-modified-id="Exemplos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exemplos</a></span><ul class="toc-item"><li><span><a href="#Fase-I-+-Fase-II" data-toc-modified-id="Fase-I-+-Fase-II-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Fase I + Fase II</a></span><ul class="toc-item"><li><span><a href="#Fase-I:-Testa-se-a-solução-é-factível-(e-obtém-uma-base)" data-toc-modified-id="Fase-I:-Testa-se-a-solução-é-factível-(e-obtém-uma-base)-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Fase I: Testa se a solução é factível (e obtém uma base)</a></span></li><li><span><a href="#Testando-a-solução-encontrada" data-toc-modified-id="Testando-a-solução-encontrada-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Testando a solução encontrada</a></span></li></ul></li><li><span><a href="#Somente-Fase-II-(base-já-fornecida-quando-instanciamos-o-algoritmo)" data-toc-modified-id="Somente-Fase-II-(base-já-fornecida-quando-instanciamos-o-algoritmo)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Somente Fase II (base já fornecida quando instanciamos o algoritmo)</a></span><ul class="toc-item"><li><span><a href="#Testando-a-solução-encontrada" data-toc-modified-id="Testando-a-solução-encontrada-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Testando a solução encontrada</a></span></li></ul></li></ul></li></ul></div>

## Resumo

Construção do algoritmo Simplex de duas fases:

**Fase I: Busca de uma solução factível**
A função `find_basis` chamada em `Simplex.fase_1` (mas pode sser chamada de fora da classe também) faz a verificação de "factibilidade" do problema, caso positivo, fornece uma base para iniciar a busca de uma solução ótima.

**Fase II: Busca de uma solução ótima**
A função `Simplex.fase_2`* busca uma solução ótima para o problema, dada uma base.


\* Essa função também é usada na fase anterior para resolver o "problema auxiliar" que criamos para achar uma base para o problema original.

## Implementação

In [1]:
import numpy as np
from numpy.linalg import inv

def find_basis(A, c, b):
    """
    Gera uma base para o problema, determinando se o problema é factível.

    Return
    ------
    B : np.array
        Base para o PL.
    x : np.array
        Solução básica associada a B.
    """

    print("Buscando uma base...")

    # Criamos i novas variáveis
    i = A.shape[0]
    print('Expansão da matriz A:')
    A_exp = np.append(A, np.identity(i), axis=1)
    print(A_exp)

    print('Atualizando c e criando a base inicial:')
    c = np.zeros((A_exp.shape[1], 1))
    c[-i:] = -1
    B = np.where(c == -1)[0]
    print('c: \n', c)
    print('B: ', B)

    print('Solução inicial:')
    #pinv = np.linalg.pinv(A_exp)
    #x = np.dot(pinv, b)
    x = np.zeros(A_exp.shape[1])
    x[B] = np.dot(A_exp[:, B], b).reshape(B.shape[0])
    print(x)

    # Calculando simplex para o problema "artificial"
    s = Simplex(A_exp, c, b, B=B, maximum=False)
    x = s.fase_2()

    # Checando solução factível: variáveis novas devem estar zeradas!
    if np.all(x[-i:] == 0):
        B = np.where(x[:-i] != 0)[0]
        
        x = np.zeros(A.shape[1])
        x[B] = np.dot(A_exp[:, B], b).reshape(B.shape[0])
        x = x.reshape(x.shape[0], 1)
        
        print(">> Base encontrada: \nB = \n{}".format(B))
        print(">> Solução básica: \nx = \n{}".format(x))
        return B

    else:
        print("Problema não é factível :(")


class Simplex:

    """
    Instancia o metodo Simplex.

    Parameters
    ----------
    A : np.matrix
        Matriz de restricoes do PL
    c : np.array
        Vetor de custos do PL.
    b : np.array
        Vetor de resposta do PL.
    B : np.array, optional
        Base do PL (pode ser encontrada na fase 1).
    maximum : bool
        Define problema de maximização (True) ou minimização (False). Defautl True.
    w : int
        Custo independente do PL.
    steps : int, optional
        Número máximo de iterações. Default 100.

    Return
    ------
    x : np.array
        Solucao otima do PL (nx1).
    """
    def __init__(self, A, c, b, B=np.nan, maximum=True, w=0, steps=500):

        self.A = A
        self.c = c
        self.b = b
        self.w = w
        self.steps = steps
        
        if type(B) == np.ndarray:
            self.B = B
            
        if maximum:
            self.operator = max
        else:
            self.operator = min
            
    def canonical_form(self):
        """
        Passa o PL para a forma canônica: (i) A_B deve ser identidade, (ii) c_B = 0
        """
        
        if not np.all(self.c[self.B] != 0) or not np.all(self.A[:, self.B] != np.identity(len(self.B))):
            
            A_Binv = np.linalg.inv(self.A[:, self.B])
            self.y = A_Binv.T.dot(self.c[self.B])
            self.c = self.c.T - self.y.T.dot(self.A).ravel()
            self.c = self.c.T
            self.w = self.w + np.dot(self.y.T, self.b)
            
            self.A = A_Binv.dot(self.A)
            self.b = A_Binv.dot(self.b)
            
            self.x = np.zeros(self.A.shape[1])
            self.x[self.B] = np.dot(self.A[:,self.B],self.b).reshape(self.B.shape[0])
            
            print('A: \n', self.A.round(2))
            print('b: \n', self.b.round(2))
            print('c: \n', self.c.round(2))
            print('w: \n', self.w)
            print(">> Problema canonizado! x: ", self.x)
            
        else:
            print(">> Problema ja na forma canonica! x: ", self.x)
    
    def step(self):
        
        x = np.zeros(self.A.shape[1])
        x[self.B] = np.linalg.solve(self.A[:, self.B], self.b).T
        
        # Encontrar tamanho do passo
        rows = self.b.shape[0]
        
        t_set = np.array([self.b[i]/self.A[i, self.k] for i in range(rows)])
        #t = t_set[t_set > 0] # t > 0
        t_set = t_set[t_set > 0] # t > 0
        print('t: {}'.format(t_set))
        
        if self.operator:
            t = min(t_set) # problema de maximizacao
        else:
            t = max(t_set) # problema de minimizacao
            
        self.r = np.where(t_set == t)[0][0]
        print('r: {}'.format(self.r))

        # Dar o passo escolhido: x_B = b - t*Ak
        x_B = self.b - t*self.A[:, self.k].reshape(self.A.shape[0], 1)
        x_B = np.array(x_B.tolist())
        
        self.x = np.zeros((self.c.shape[0], 1))
        self.x[self.k] = t
        
        self.x[self.B] = x_B
        print('>> x: \n{}'.format(self.x))

    def fase_1(self):
        
        print('\n\nFASE 1:')
        self.B = find_basis(self.A,  self.c, self.b)
        
    def fase_2(self):
        
        print('\n\nFASE II:')
        self.N = list(set(range(self.A.shape[1])) - set(self.B))

        i = 0
        while i < self.steps:
            print("\n>>> Iteração {}:".format(i))
            print("\n1. Reescrevendo o PL na forma canonica para a base B...\n")
            self.canonical_form()
            
            print("\n2. Teste de otimalidade (c_N <= 0):")
            if np.all(self.c[self.N] <= 0):
                print(">>> Processo terminado!")
                return self.x
            print("Não passou no teste...\n")
            print(">> c_N: \n{}".format(self.c[self.N]))

            print("\n3. Seleciona k em N tal que c[k] > 0...")
            #print(np.where(self.c > 0))
            self.k = np.where(self.c > 0.0001)[0][0] # primeiro valor não nulo
            print(">> k: {}".format(self.k))
            
            print("\n4. Teste de unboundness (A[:,k] <= 0):")
            if np.all(self.A[:, self.k] <= 0):
                (">>> Processo terminado! O problema é ilimitado :(")
                return
            print("Não passou no teste...\n")
            print(">> A[:,k]: \n{}".format(self.A[:, self.k].round(2)))
            
            print("\n5. Passo Simplex...")
            self.step()

            print("\n6. Atualizando a base...")
            print(self.B)
            self.B = np.append(self.B, self.k)
            print('Adicionando o elemento k (k = {}):'.format(self.k))
            print(self.B)
            self.B = np.delete(self.B, self.r)
            print('Removendo o r-ésimo elemento (r = {}):'.format(self.r))
            print(self.B)
            
            self.N = list(set(range(self.A.shape[1])) - set(self.B))
            print(">> B: \n{}".format(self.B))
            
            i += 1

        print("Processo terminado!")
        return self.x

## Exemplos
---

### Fase I + Fase II

In [2]:
# Formato do PL: max{z = cx + w : Ax = b, x >= 0}
A = np.array([[1,5,2,1], 
              [-2,-9,0,3]])
c = np.array([1,2,-1,3]).reshape(4,1)
b = np.array([7,-13]).reshape(2,1)

#### Fase I: Testa se a solução é factível (e obtém uma base)

In [3]:
s = Simplex(A, c, b)
s.fase_1()



FASE 1:
Buscando uma base...
Expansão da matriz A:
[[ 1.  5.  2.  1.  1.  0.]
 [-2. -9.  0.  3.  0.  1.]]
Atualizando c e criando a base inicial:
c: 
 [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]]
B:  [4 5]
Solução inicial:
[  0.   0.   0.   0.   7. -13.]


FASE II:

>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 1.  5.  2.  1.  1.  0.]
 [-2. -9.  0.  3.  0.  1.]]
b: 
 [[  7.]
 [-13.]]
c: 
 [[-1.]
 [-4.]
 [ 2.]
 [ 4.]
 [ 0.]
 [ 0.]]
w: 
 [[6.]]
>> Problema canonizado! x:  [  0.   0.   0.   0.   7. -13.]

2. Teste de otimalidade (c_N <= 0):
Não passou no teste...

>> c_N: 
[[-1.]
 [-4.]
 [ 2.]
 [ 4.]]

3. Seleciona k em N tal que c[k] > 0...
>> k: 2

4. Teste de unboundness (A[:,k] <= 0):
Não passou no teste...

>> A[:,k]: 
[2. 0.]

5. Passo Simplex...
t: [3.5]
r: 0
>> x: 
[[  0. ]
 [  0. ]
 [  3.5]
 [  0. ]
 [  0. ]
 [-13. ]]

6. Atualizando a base...
[4 5]
Adicionando o elemento k (k = 2):
[4 5 2]
Removendo o r-ésimo elemento (r = 0):
[5 2]
>> B: 
[5 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:137: RuntimeWarning: divide by zero encountered in true_divide


In [4]:
x = s.fase_2()



FASE II:

>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 0.83  4.    1.    0.  ]
 [-0.67 -3.    0.    1.  ]]
b: 
 [[ 5.67]
 [-4.33]]
c: 
 [[ 3.83]
 [15.  ]
 [ 0.  ]
 [ 0.  ]]
w: 
 [[-18.66666667]]
>> Problema canonizado! x:  [ 0.          0.          5.66666667 -4.33333333]

2. Teste de otimalidade (c_N <= 0):
Não passou no teste...

>> c_N: 
[[ 3.83333333]
 [15.        ]]

3. Seleciona k em N tal que c[k] > 0...
>> k: 0

4. Teste de unboundness (A[:,k] <= 0):
Não passou no teste...

>> A[:,k]: 
[ 0.83 -0.67]

5. Passo Simplex...
t: [6.8 6.5]
r: 1
>> x: 
[[6.5 ]
 [0.  ]
 [0.25]
 [0.  ]]

6. Atualizando a base...
[2 3]
Adicionando o elemento k (k = 0):
[2 3 0]
Removendo o r-ésimo elemento (r = 1):
[2 0]
>> B: 
[2 0]

>>> Iteração 1:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[-0.    0.25  1.    1.25]
 [ 1.    4.5   0.   -1.5 ]]
b: 
 [[0.25]
 [6.5 ]]
c: 
 [[ 0.  ]
 [-2.25]
 [ 0.  ]
 [ 5.75]]
w: 
 [[6.25]]
>> Problema canonizado! x

#### Testando a solução encontrada

In [5]:
x

array([[6.8],
       [0. ],
       [0. ],
       [0.2]])

In [6]:
np.dot(A, x)

array([[  7.],
       [-13.]])

In [7]:
b # Ax = b!

array([[  7],
       [-13]])

Checando:

In [8]:
from scipy.optimize import linprog
linprog(-c, A_eq=A, b_eq=b) # usando -c pois o método é de minimização!

     con: array([ 8.88178420e-16, -1.77635684e-15])
     fun: -7.399999999999999
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([6.8, 0. , 0. , 0.2])

### Somente Fase II (base já fornecida quando instanciamos o algoritmo)

In [9]:
# Formato do PL: max{z = cx + w : Ax = b, x >= 0}
A = np.array([[1,1,1,0,0], 
               [2,1,0,1,0], 
               [-1,1,0,0,1]])
c = np.array([2,3,0,0,0]).reshape(5,1)
b = np.array([6,10,4]).reshape(3,1)
#B = np.array([2,3,4])

s = Simplex(A, c, b)

In [10]:
# Fase I: Testa se a solução é factível
s.fase_1()



FASE 1:
Buscando uma base...
Expansão da matriz A:
[[ 1.  1.  1.  0.  0.  1.  0.  0.]
 [ 2.  1.  0.  1.  0.  0.  1.  0.]
 [-1.  1.  0.  0.  1.  0.  0.  1.]]
Atualizando c e criando a base inicial:
c: 
 [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]]
B:  [5 6 7]
Solução inicial:
[ 0.  0.  0.  0.  0.  6. 10.  4.]


FASE II:

>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 1.  1.  1.  0.  0.  1.  0.  0.]
 [ 2.  1.  0.  1.  0.  0.  1.  0.]
 [-1.  1.  0.  0.  1.  0.  0.  1.]]
b: 
 [[ 6.]
 [10.]
 [ 4.]]
c: 
 [[2.]
 [3.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]]
w: 
 [[-20.]]
>> Problema canonizado! x:  [ 0.  0.  0.  0.  0.  6. 10.  4.]

2. Teste de otimalidade (c_N <= 0):
Não passou no teste...

>> c_N: 
[[2.]
 [3.]
 [1.]
 [1.]
 [1.]]

3. Seleciona k em N tal que c[k] > 0...
>> k: 0

4. Teste de unboundness (A[:,k] <= 0):
Não passou no teste...

>> A[:,k]: 
[ 1.  2. -1.]

5. Passo Simplex...
t: [6. 5.]
r: 1
>> x: 
[[5.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [

In [11]:
# Fase II: Acha a solução ótima
x = s.fase_2()



FASE II:

>>> Iteração 0:

1. Reescrevendo o PL na forma canonica para a base B...

A: 
 [[ 1.   0.   0.5  0.  -0.5]
 [ 0.   1.   0.5  0.   0.5]
 [ 0.   0.  -1.5  1.   0.5]]
b: 
 [[1.]
 [5.]
 [3.]]
c: 
 [[ 0. ]
 [ 0. ]
 [-2.5]
 [ 0. ]
 [-0.5]]
w: 
 [[17.]]
>> Problema canonizado! x:  [1. 5. 0. 3. 0.]

2. Teste de otimalidade (c_N <= 0):
>>> Processo terminado!


#### Testando a solução encontrada

In [12]:
x

array([1., 5., 0., 3., 0.])

In [13]:
np.dot(A, x)

array([ 6., 10.,  4.])

In [14]:
np.dot(c.T, x)

array([17.])

Checando:

In [15]:
from scipy.optimize import linprog
linprog(-c, A_eq=A, b_eq=b) # usando -c pois o método é de minimização!

     con: array([0., 0., 0.])
     fun: -17.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1., 5., 0., 3., 0.])